In [106]:
from os import getcwd, walk
from pandas import read_csv, concat
from plotnine import ggplot, aes, geom_line, theme_bw

In [107]:
data_sets = []  # List for storing the data set locations
data_sets_directory = f"{getcwd()}/datasets" # Home directory of the data 
for (root, dirs, files) in walk(data_sets_directory):    # Looping over directories 
    for file in files:  # Looping through the files 
        file_path = root + "/" + file   # Setting up the path 
        data_sets.append(file_path)  # Extending the stored directoriess

In [108]:
mixed_res = read_csv("Intcal20.csv")
mixed_res = mixed_res.assign(id="Intcal20")

In [109]:
pd_concat_iter = [] # Iterable holding DataFrames corresponding to the single year datasets 

for data_set in data_sets:
    iter_set = read_csv(data_set)
    iter_set_id = data_set.split("/")[-1].strip(".csv") # id in format AuthorYY_Species
    iter_set = iter_set.assign(id=iter_set_id)
    pd_concat_iter.append(iter_set)

annual_res = concat(pd_concat_iter) # Constructing the dataset

In [110]:
d14c = concat([mixed_res, annual_res])

,year,d14c,sig_d14c,id
0,1725.0,12.2,2.3,Intcal20
1,1724.0,9.0,2.1,Intcal20
2,1724.0,14.0,2.3,Intcal20
3,1723.0,12.0,2.3,Intcal20
4,1722.0,11.2,2.1,Intcal20
...,...,...,...,...
32,-637.0,3.1,1.7,Sakurai20_CedarLw
33,-636.0,6.5,1.7,Sakurai20_CedarLw
34,-635.0,2.4,1.7,Sakurai20_CedarLw
35,-634.0,1.8,2.4,Sakurai20_CedarLw


Pick a 100 year block and test the code. So I could fill in the 25 years before and 25 years after with the Intcal20 interpolated curve. Take all of Intcal20 and place the actual data into the interpolated data. This would require rescaling the data with a linear curve or something along those lines. Only test the injection recovery at te real years. 

Linear combination of the basis vectors provided by the template, linear trend and constant.
$Ax = b$ simple linear regression with a 50 by 3 design matrix that is dotted with a 3 long position vector to produce the predicted d14c. The first row is just the template. The second is just rows and the final is just the year.

Resample the data with gaussian noise of the same magnitude as the error bars. Plot the amplitdues as a histogram. Calculate the chi squared statsitic and then do just trend and mean and calculate the difference in chi squared statistic. I choose the significance threshold of the resulting distribution.This gives the false positive rate.

Then you inject a number of small events and see if they are detected based on the thresholds you decided. This gives you the false negative rate. 

Repeating this for every year I can plot the year vs the 50% false positive amplitude of the event. Interpolate over all of the years